In [14]:
import pandas as pd
import geopandas as gpd
from shapely import Point

In [2]:
df = pd.read_csv("../data/Counterglow+Facility+List+Complete.csv")

In [3]:
df.head()

,Name,Lat,Lat.1,Address,City,State,County,Description,Business/company name,Postal address,Phone number,Region,Facility name,Number of animals,Full address,Website URL,Postcode,Suburb/city,Contracted to,Farm Type
0,78,43.396729,-95.923149,"1757 Lily Avenue George 51237, United States",NaN,IA,Lyon County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pigs (Meat)
1,84,42.925350,-96.429291,"1804 500th Street Hawarden 51023, United States",NaN,IA,Sioux County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pigs (Meat)
2,87,40.658218,-92.410202,"21166 Mahogany Avenue Bloomfield 52537, United...",NaN,IA,Davis County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pigs (Meat)
3,88,40.643219,-92.409889,"27268 Mahogany Avenue Bloomfield 52537, United...",NaN,IA,Davis County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pigs (Meat)
4,89,40.660851,-92.421219,"21166 280th Street Bloomfield 52537, United St...",NaN,IA,Davis County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pigs (Meat)


In [4]:
len(df)

29853

In [20]:
df["Farm Type"].unique()

array(['Pigs (Meat)', 'Cows (Meat)',
       'Cows, Bulls, Horses, Other Animals - Rodeos', 'Unknown',
       'Cows (Dairy, Meat)', nan,
       'Mink/Foxes/Rabbits/Other Mammals (Fur)', 'Slaughterhouses',
       'Chickens (Meat)', 'Chickens (Eggs)',
       'Fish & Marine Animals (Meat)', 'Goats (Meat)', 'Turkeys (Meat)',
       'Live Animal Saleyards', 'Alligators & Crocodiles (Skin, Meat)',
       'Sheep (Meat)', 'Horse Racetracks', 'Scientific Testing Labs',
       'Miscellaneous', 'Corporate Office/Hq',
       'Chickens & Other Birds (Meat)', 'Honey Bees (Honey)',
       'Live Animal Export', 'Rabbits (Fur, Meat)', 'Zoos',
       'Ducks (Meat)', 'Horses',
       'Rendering (Processing leftover animal body parts from slaughterhouses)',
       'Pullets - Birds (Meat)', 'Pigs (Meat) & Cows (Dairy, Meat)',
       'Greyhound Racetracks', 'Quails (Meat)',
       'Multiple Types of Animals', 'Koalas',
       'Deer (Hunting and/or Butchering)', 'Circuses',
       'Pigs & Cows (Meat)', 'Mice 

In [21]:
df_poultry = df[(df["Farm Type"] == "Chickens (Meat)") | (df["Farm Type"] == "Chickens & Other Birds (Meat)")]

In [22]:
len(df_poultry)

2735

In [27]:
len(df_poultry[df_poultry.State == "AR"])

22

In [40]:
df_poultry.State.unique()

array(['IA', 'AL', 'NE', 'MS', 'TX', 'IN', 'IL', 'NC', 'MD', 'MO', 'PA',
       'OH', 'WI', 'ID', 'GA', 'OK', 'WA', 'LA', 'VA', nan, 'SD', 'CA',
       'AR', 'SC', 'MI', 'KY', 'DE', 'OR', 'TN', 'MN', 'KS', 'NJ', 'MT',
       'NY', 'ME', 'VT', 'ND', 'WV', 'CO'], dtype=object)

In [9]:
df_states = gpd.read_file("../data/all_states.geojson")

In [28]:
df_states[df_states.state == "Arkansas"]

,state,plant_access,area,geometry
4,Arkansas,1,4.062816e+10,"MULTIPOLYGON (((-94.00708 33.20157, -94.01254 ..."
5,Arkansas,2,2.535141e+10,"MULTIPOLYGON (((-94.40061 34.34164, -94.44440 ..."
6,Arkansas,3,2.082532e+10,"MULTIPOLYGON (((-90.74369 35.56358, -90.76770 ..."
7,Arkansas,4,1.228056e+10,"MULTIPOLYGON (((-94.33686 35.57354, -94.33547 ..."


In [38]:
abb2state = {"AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California","CO":"Colorado","CT":"Connecticut","DE":"Delaware","FL":"Florida","GA":"Georgia","HI":"Hawaii","ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana","ME":"Maine","MD":"Maryland","MA":"Massachusetts","MI":"Michigan","MN":"Minnesota","MS":"Mississippi","MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey","NM":"New Mexico","NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma","OR":"Oregon","PA":"Pennsylvania","RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota","TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington","WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"}


In [45]:
# geojson has full state names and Counterglow data has state abbreviations
list_of_farms = []
for state in df_poultry.State.unique():
    for i, farm in df_poultry[df_poultry.State == state].iterrows():
        pt = Point(farm["Lat.1"], farm["Lat"])
        for k, area in df_states[df_states.state == abb2state[state]].iterrows():
            if area.geometry.contains(pt):
                farm_data = {
                    "state": abb2state[state],
                    "geometry": pt,
                    "address": farm.Address,
                    "company": farm["Business/company name"]
                }
                print(farm_data)
                list_of_farms.append(farm_data)

{'state': 'Alabama', 'geometry': <POINT (-85.341 33.279)>, 'address': 'Roanoke 36274, United States', 'company': nan}
{'state': 'Mississippi', 'geometry': <POINT (-90.789 31.699)>, 'address': 'Unnamed Road Union Church 39668, United States', 'company': nan}
{'state': 'Maryland', 'geometry': <POINT (-75.641 38.003)>, 'address': '4461 Shelltown Road Marion Station 21838, United States', 'company': nan}
{'state': 'Missouri', 'geometry': <POINT (-94.102 36.777)>, 'address': '20183 Wolf Drive Fairview 64842, United States', 'company': nan}
{'state': 'Pennsylvania', 'geometry': <POINT (-77.175 40.683)>, 'address': '307 End Road Richfield 17086, United States', 'company': nan}
{'state': 'Georgia', 'geometry': <POINT (-85.344 33.688)>, 'address': 'Tallapoosa 30176, United States', 'company': nan}
{'state': 'Oklahoma', 'geometry': <POINT (-94.751 36.894)>, 'address': '8500 South 630 Road Miami 74354, United States', 'company': nan}
{'state': 'Louisiana', 'geometry': <POINT (-92.48 32.947)>, 'ad

In [46]:
contained_poultry_farms = gpd.GeoDataFrame(list_of_farms)

In [47]:
contained_poultry_farms

,state,geometry,address,company
0,Alabama,POINT (-85.34119 33.27888),"Roanoke 36274, United States",NaN
1,Mississippi,POINT (-90.78935 31.69891),"Unnamed Road Union Church 39668, United States",NaN
2,Maryland,POINT (-75.64085 38.00333),"4461 Shelltown Road Marion Station 21838, Unit...",NaN
3,Missouri,POINT (-94.10210 36.77681),"20183 Wolf Drive Fairview 64842, United States",NaN
4,Pennsylvania,POINT (-77.17497 40.68262),"307 End Road Richfield 17086, United States",NaN
5,Georgia,POINT (-85.34396 33.68792),"Tallapoosa 30176, United States",NaN
6,Oklahoma,POINT (-94.75120 36.89411),"8500 South 630 Road Miami 74354, United States",NaN
7,Louisiana,POINT (-92.48014 32.94734),NaN,NaN
8,Virginia,POINT (-79.07302 38.38982),"26 Millers Road Bridgewater 22812, United States",NaN
9,California,POINT (-120.86532 37.71307),"5012 Bentley Road Oakdale 95361, United States",NaN
